In [1]:
import pandas as pd
from google.colab import drive
drive.mount("/content/gdriv")

Mounted at /content/gdriv


In [2]:
import os
data_path = '/content/gdriv/MyDrive/산데과'
data_list = os.listdir(data_path)
train_dt = pd.read_csv(data_path+'/'+data_list[9]) 
test_dt = pd.read_csv(data_path+'/'+data_list[10]) 

In [3]:
train_dt.head()

,PM,PM2,EG1,EG2,EG3,EG4,EG5,EG6,EG7,EG8,EG9,EG10,EG11,EG12,EG13,EG14,EG15,EG16,EG17,EG18,EG19,EG20,HUM,TMP,WD,WS
0,27.0,25.0,1305.26,15069.82,12815.28,2847.84,2847.84,1230.46,14206.22,12080.88,2684.64,2684.64,291.72,3368.04,2864.16,636.48,636.48,583.44,6736.08,5728.32,1272.96,1272.96,91,7.2,0.906308,0.6
1,52.0,17.0,1021.02,11788.14,10024.56,2227.68,2227.68,729.30,8420.10,7160.40,1591.20,1591.20,0.00,0.00,0.00,0.00,0.00,437.58,5052.06,4296.24,954.72,954.72,36,14.3,0.438371,1.3
2,22.0,12.0,867.68,10017.76,8519.04,1893.12,1893.12,1301.52,15026.64,12778.56,2839.68,2839.68,145.86,1684.02,1432.08,318.24,318.24,291.72,3368.04,2864.16,636.48,636.48,38,6.7,0.992546,0.5
3,23.0,13.0,729.30,8420.10,7160.40,1591.20,1591.20,729.30,8420.10,7160.40,1591.20,1591.20,0.00,0.00,0.00,0.00,0.00,145.86,1684.02,1432.08,318.24,318.24,23,9.7,0.241922,3.0
4,90.0,63.0,946.22,10924.54,9290.16,2064.48,2064.48,575.96,6649.72,5654.88,1256.64,1256.64,0.00,0.00,0.00,0.00,0.00,583.44,6736.08,5728.32,1272.96,1272.96,18,5.9,0.484810,1.3


In [4]:
test_dt.head()

,EG1,EG2,EG3,EG4,EG5,EG6,EG7,EG8,EG9,EG10,EG11,EG12,EG13,EG14,EG15,EG16,EG17,EG18,EG19,EG20,HUM,TMP,WD,WS
0,583.44,6736.08,5728.32,1272.96,1272.96,583.44,6736.08,5728.32,1272.96,1272.96,437.58,5052.06,4296.24,954.72,954.72,437.58,5052.06,4296.24,954.72,954.72,68,23.7,0.882948,0.7
1,875.16,10104.12,8592.48,1909.44,1909.44,1151.92,13299.44,11309.76,2513.28,2513.28,437.58,5052.06,4296.24,954.72,954.72,583.44,6736.08,5728.32,1272.96,1272.96,34,3.5,0.190809,3.5
2,729.30,8420.10,7160.40,1591.20,1591.20,1092.08,12608.56,10722.24,2382.72,2382.72,0.00,0.00,0.00,0.00,0.00,437.58,5052.06,4296.24,954.72,954.72,58,11.9,0.754710,0.6
3,875.16,10104.12,8592.48,1909.44,1909.44,1600.72,18481.04,15716.16,3492.48,3492.48,437.58,5052.06,4296.24,954.72,954.72,291.72,3368.04,2864.16,636.48,636.48,25,7.0,0.275637,1.9
4,1237.94,14292.58,12154.32,2700.96,2700.96,1806.42,20855.94,17735.76,3941.28,3941.28,291.72,3368.04,2864.16,636.48,636.48,1301.52,15026.64,12778.56,2839.68,2839.68,66,13.7,-0.173648,0.4


In [5]:
dt=pd.concat([train_dt,test_dt])

In [6]:
idx0 = train_dt.loc[train_dt['PM2']<=15,:].index
idx1 = train_dt.loc[(train_dt['PM2']>=15)&(train_dt['PM2']<=35),:].index
idx2 = train_dt.loc[train_dt['PM2']>=36,:].index

train_dt.iloc[idx0,1]=0
train_dt.iloc[idx1,1]=1
train_dt.iloc[idx2,1]=2

train_dt['PM2'].value_counts()

1.0    1670
2.0    1033
0.0     725
Name: PM2, dtype: int64

In [7]:
idx0 = train_dt.loc[train_dt['PM2']<=30,:].index
idx1 = train_dt.loc[(train_dt['PM2']>=30)&(train_dt['PM2']<=80),:].index
idx2 = train_dt.loc[train_dt['PM2']>=81,:].index

train_dt.iloc[idx0,0]=0
train_dt.iloc[idx1,0]=1
train_dt.iloc[idx2,0]=2

train_dt['PM2'].value_counts()

1.0    1670
2.0    1033
0.0     725
Name: PM2, dtype: int64

In [8]:
x=dt.iloc[:,2:]
pm2=train_dt.iloc[:,1]
pm1=train_dt.iloc[:,1]

In [9]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(x)
x = min_max_scaler.transform(x)
x

array([[0.64510166, 0.64510166, 0.64510166, ..., 0.4469914 , 0.95315033,
        0.10909091],
       [0.50462107, 0.50462107, 0.50462107, ..., 0.6504298 , 0.71916416,
        0.23636364],
       [0.42883549, 0.42883549, 0.42883549, ..., 0.43266476, 0.99627282,
        0.09090909],
       ...,
       [0.50462107, 0.50462107, 0.50462107, ..., 0.48137536, 0.75750058,
        0.23636364],
       [0.2883549 , 0.2883549 , 0.2883549 , ..., 0.32091691, 0.68727949,
        0.10909091],
       [0.75231054, 0.75231054, 0.75231054, ..., 0.58739255, 0.97275723,
        0.16363636]])

In [10]:
train_x=pd.DataFrame(x).iloc[:3428,:]
test_x = pd.DataFrame(x).iloc[3429:]

In [11]:
from sklearn.model_selection import train_test_split
pm2_trainx,pm2_testx,pm2_trainy,pm2_testy = train_test_split(train_x,pm2,test_size=0.3,random_state=15)

In [12]:
from sklearn.model_selection import train_test_split
pm1_trainx,pm1_testx,pm1_trainy,pm1_testy = train_test_split(train_x,pm1,test_size=0.3,random_state=15)

In [13]:
pm2_trainx=pm2_trainx.to_numpy()
pm2_testx=pm2_testx.to_numpy()

In [14]:
pm1_trainx=pm1_trainx.to_numpy()
pm1_testx=pm1_testx.to_numpy()

In [15]:
pm2_testy

2690    2.0
2978    1.0
2984    0.0
670     2.0
2845    1.0
       ... 
1032    2.0
3140    0.0
1706    2.0
2049    1.0
659     1.0
Name: PM2, Length: 1029, dtype: float64

#pm2

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [48]:
class mlp(nn.Module):

  def __init__(self,input_size):
    super(mlp,self).__init__()
    self.input_size = input_size
    self.fc1 = nn.Linear(self.input_size,18)
    self.bn1 = torch.nn.BatchNorm1d(18)
    self.fc2 = nn.Linear(18,9)
    self.bn2 = torch.nn.BatchNorm1d(9)
    self.fc4 = nn.Linear(9,3)
    self.relu = torch.nn.ReLU()
    self.dropout = torch.nn.Dropout()
    self.sf=torch.nn.Softmax()

  def forward(self,x):#Linear -> BatchNorm -> function -> Dropout 순서로 구성
    output = self.fc1(x)
    output = self.bn1(output)
    output = self.relu(output)
    output = self.dropout(output)
    output = self.fc2(output)
    output = self.bn2(output)
    output = self.relu(output)
    output = self.dropout(output)
    output = self.fc4(output)
    output = self.sf(output)
    return output

In [42]:
pm2_trainx = torch.FloatTensor(pm2_trainx)
pm2_testx = torch.FloatTensor(pm2_testx)

In [37]:
pm2_trainy = torch.FloatTensor(pm2_trainy.values)
pm2_trainy = pm2_trainy.long()# output이 정수여야 하기에 long으로 casting해 줌

pm2_testy = torch.FloatTensor(pm2_testy.values)
pm2_testy = pm2_testy.long()

TypeError: ignored

In [49]:
input_size = pm2_trainx.size()[1]
model = mlp(input_size)

In [50]:
optimizer = optim.Adam(model.parameters(),lr=0.0001)#lr = learning rate(gradient descent에서 그 lambda값)
loss = nn.CrossEntropyLoss()#다중분류 문제이기 때문에

In [51]:
for e in range(15000):#epoch 즉 학습 횟수를 5000으로 지정하겠다
  optimizer.zero_grad()#optimizer를 zero로 초기화 시켜줌
  out = model(pm2_trainx)
  L = loss(out,pm2_trainy)
  L.backward()#역전파 과정
  optimizer.step()#파라미터를 최적화 시키는 과정
  if (e+1) % 500 == 0:#epoch이 500일 때 마다 모델을 평가해주는 부분
    with torch.no_grad():#테스트 할 때는 변형하지 않기 위하여 추가해주는 함수
      model.eval
      pred = model(pm2_testx)
      acc = sum(torch.max(pred,1)[1] == pm2_testy).numpy()/len(pm2_testy)#torch.max[0]=확률을,[1]=indicator를 반환
    print('[Epoch: {}/{}] [Loss: {:2f}] [Test Accuracy: {:2f}]'.format(e+1,15000,L.item(),acc*100))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch: 500/5000] [Loss: 1.075810] [Test Accuracy: 46.550049]
[Epoch: 1000/5000] [Loss: 1.048351] [Test Accuracy: 50.242954]
[Epoch: 1500/5000] [Loss: 1.031502] [Test Accuracy: 52.478134]
[Epoch: 2000/5000] [Loss: 1.021776] [Test Accuracy: 53.741497]
[Epoch: 2500/5000] [Loss: 1.008969] [Test Accuracy: 53.158406]
[Epoch: 3000/5000] [Loss: 1.011169] [Test Accuracy: 50.534500]
[Epoch: 3500/5000] [Loss: 1.002991] [Test Accuracy: 54.324587]
[Epoch: 4000/5000] [Loss: 0.996988] [Test Accuracy: 54.324587]
[Epoch: 4500/5000] [Loss: 0.989703] [Test Accuracy: 55.102041]
[Epoch: 5000/5000] [Loss: 0.992004] [Test Accuracy: 54.324587]
[Epoch: 5500/5000] [Loss: 0.990329] [Test Accuracy: 54.033042]
[Epoch: 6000/5000] [Loss: 0.985935] [Test Accuracy: 54.324587]
[Epoch: 6500/5000] [Loss: 0.980741] [Test Accuracy: 54.518950]
[Epoch: 7000/5000] [Loss: 0.976875] [Test Accuracy: 56.365403]
[Epoch: 7500/5000] [Loss: 0.986214] [Test Accuracy: 53.838678]
[Epoch: 8000/5000] [Loss: 0.980143] [Test Accuracy: 53.7

In [52]:
test_x_=test_x.to_numpy()
test_x_=torch.FloatTensor(test_x_)

In [67]:
pred = model(test_x_)
result=pd.DataFrame(torch.max(pred,1)[1])
result.to_csv("201711514 박주호 pm2.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


#pm1

In [54]:
pm1_trainx = torch.FloatTensor(pm1_trainx)
pm1_testx = torch.FloatTensor(pm1_testx)

In [55]:
pm1_trainy = torch.FloatTensor(pm1_trainy.values)
pm1_trainy = pm1_trainy.long()# output이 정수여야 하기에 long으로 casting해 줌

pm1_testy = torch.FloatTensor(pm1_testy.values)
pm1_testy = pm1_testy.long()

TypeError: ignored

In [56]:
model_2 = mlp(input_size)

In [60]:
optimizer = optim.Adam(model_2.parameters(),lr=0.0001)#lr = learning rate(gradient descent에서 그 lambda값)
loss = nn.CrossEntropyLoss()#다중분류 문제이기 때문에

In [61]:
for e in range(15000):#epoch 즉 학습 횟수를 5000으로 지정하겠다
  optimizer.zero_grad()#optimizer를 zero로 초기화 시켜줌
  out = model(pm1_trainx)
  L = loss(out,pm1_trainy)
  L.backward()#역전파 과정
  optimizer.step()#파라미터를 최적화 시키는 과정
  if (e+1) % 500 == 0:#epoch이 500일 때 마다 모델을 평가해주는 부분
    with torch.no_grad():#테스트 할 때는 변형하지 않기 위하여 추가해주는 함수
      model.eval
      pred = model(pm1_testx)
      acc = sum(torch.max(pred,1)[1] == pm1_testy).numpy()/len(pm1_testy)#torch.max[0]=확률을,[1]=indicator를 반환
    print('[Epoch: {}/{}] [Loss: {:2f}] [Test Accuracy: {:2f}]'.format(e+1,15000,L.item(),acc*100))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch: 500/15000] [Loss: 0.965841] [Test Accuracy: 54.616132]
[Epoch: 1000/15000] [Loss: 0.965168] [Test Accuracy: 57.337221]
[Epoch: 1500/15000] [Loss: 0.964866] [Test Accuracy: 57.823129]
[Epoch: 2000/15000] [Loss: 0.965851] [Test Accuracy: 54.907677]
[Epoch: 2500/15000] [Loss: 0.970392] [Test Accuracy: 55.976676]
[Epoch: 3000/15000] [Loss: 0.969179] [Test Accuracy: 56.073858]
[Epoch: 3500/15000] [Loss: 0.961632] [Test Accuracy: 56.559767]
[Epoch: 4000/15000] [Loss: 0.965769] [Test Accuracy: 56.851312]
[Epoch: 4500/15000] [Loss: 0.975557] [Test Accuracy: 56.268222]
[Epoch: 5000/15000] [Loss: 0.971241] [Test Accuracy: 56.268222]
[Epoch: 5500/15000] [Loss: 0.965815] [Test Accuracy: 56.365403]
[Epoch: 6000/15000] [Loss: 0.966613] [Test Accuracy: 56.948494]
[Epoch: 6500/15000] [Loss: 0.971380] [Test Accuracy: 57.434402]
[Epoch: 7000/15000] [Loss: 0.966402] [Test Accuracy: 57.920311]
[Epoch: 7500/15000] [Loss: 0.969388] [Test Accuracy: 56.073858]
[Epoch: 8000/15000] [Loss: 0.978073] [Tes

In [68]:
pred = model(test_x_)
result_pm1=pd.DataFrame(torch.max(pred,1)[1])
result_pm1.to_csv("201711514 박주호 pm1.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
